In [86]:
import pandas as pd
from statistics import mean
from nltk.tokenize import word_tokenize

In [2]:
#imports the data
roughDat= pd.read_csv("PlayDat.csv")

In [3]:
#removes unnecessary columns.
roughDat = roughDat.drop(columns=['Unnamed: 0', 'articleHeadline', 'claimId',
       'articleId', 'articleHeadlineStance'])

In [4]:
import os
os.environ["CORENLP_HOME"] = r'D:\\Downloads\\stanford-corenlp-full-2018-10-05\\stanford-corenlp-full-2018-10-05'

In [33]:
#import re 
mylist=roughDat["claimHeadline"].unique().tolist()
#myshortlist = [re.sub(r'[^\w\s\']','',s) for s in mylist] for removing punctuation. 

In [51]:
from stanfordnlp.server import CoreNLPClient
rootIndex =[]
rootDistances = []
with CoreNLPClient(annotators=['depparse'], timeout=60000, memory='16G') as client:
    # submit the request to the server
    for sentences in mylist:

        ann = client.annotate(sentences)

        # get the first sentence
        sentence = ann.sentence[0]
        # get the dependency parse of the first sentence
        dependency_parse = sentence.basicDependencies
        root=int(dependency_parse.root[0])
        rootIndex.append(root)
        neg_Pos=None
        for i in dependency_parse.edge:
            if i.dep=="neg":
                neg_Pos=int(i.target)
        if neg_Pos!=None:
            rootDist=abs(root - neg_Pos)
        else:
            rootDist=0
        rootDistances.append(rootDist)        

Starting server with command: java -Xmx16G -cp D:\\Downloads\\stanford-corenlp-full-2018-10-05\\stanford-corenlp-full-2018-10-05/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-3bf3696d73c04710.props -preload depparse


In [52]:
rootDistances

[15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15]

In [53]:
rootIndex

[3, 5, 2, 5, 5, 3, 4, 3, 2, 5, 4, 3, 6, 2, 3, 5, 3, 3, 6, 7, 7, 2, 3, 3, 3]

In [10]:
refuting =['fake',
           'fraud',
           'hoax',
           'false',
           'deny', 
           'denies',
           'refute',
           'not',
           'despite',
           'nope',
           'doubt', 
           'doubts',
           'bogus',
           'debunk',
           'pranks',
           'prank',
           'retract']
hedging = [
        'alleged', 'allegedly',
        'apparently',
        'appear', 'appears',
        'claim', 'claims',
        'could',
        'evidently',
        'largely',
        'likely',
        'mainly',
        'may', 'maybe', 'might',
        'mostly',
        'perhaps',
        'presumably',
        'probably',
        'purported', 'purportedly',
        'reported', 'reportedly',
        'rumor', 'rumour', 'rumors', 'rumours', 'rumored', 'rumoured',
        'says',
        'seem',
        'somewhat',
        'supposedly',
        'unconfirmed']

In [35]:
mytokenizedlist =[ word_tokenize(i) for i in mylist]

In [92]:
hedged_RDist =[]
k=0
for i in mytokenizedlist:
    Distances=[]
    for j in hedging:
        if j in i:
            Dist=(abs(rootIndex[k]-i.index(j)))
            Distances.append(Dist)
        else:
            Distances.append(0)
    k+=1
    Distances = [i for i in Distances if i != 0]
    if len(Distances)==0:
        Distances.append(0)
    hedged_RDist.append(mean(Distances))

20


In [93]:
len(hedged_RDist)

25

In [94]:
hedged_RDist

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20]

In [95]:
refute_RDist =[]
k=0
for i in mytokenizedlist:
    Distances=[]
    for j in refuting:
        if j in i:
            Dist=(abs(rootIndex[k]-i.index(j)))
            Distances.append(Dist)
        else:
            Distances.append(0)
    k+=1
    Distances = [i for i in Distances if i != 0]
    if len(Distances)==0:
        Distances.append(0)
    refute_RDist.append(mean(Distances))

In [99]:
refute_RDist

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]